In [ ]:
%load_ext autoreload
%autoreload 2

import os
ipy = get_ipython()
#data_dir_general = '/home/demitau/data_Quentin'
#data_subdir_mem_err_main = 'full_experiments/data2'
from os.path import join as pjoin
import os,sys; sys.path.append(os.path.expandvars('$CODE_MEMORY_ERRORS'))
import error_sensitivity
#data_dir_input = pjoin(data_dir_general,data_subdir_mem_err_main)
import pandas as pd

#data_dir_general = '/home/demitau/data_Quentin'
#data_subdir_mem_err_main = 'full_experiments/data2'
#data_dir_input = pjoin(data_dir_general,data_subdir_mem_err_main)
#scripts_dir = pjoin(data_dir_general,'full_experiments','scripts2')

data_dir_general = os.path.expandvars('$DATA_QUENTIN')
data_dir_input = os.path.expandvars('$DATA_MEMORY_ERRORS_STAB_AND_STOCH')
#scripts_dir = pjoin( os.path.expandvars('$CODE_MEMORY_ERRORS'), 'previous_analyses')
scripts_dir = pjoin( os.path.expandvars('$CODE_MEMORY_ERRORS'))



import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
print('seaborn ver = ', sns.__version__)
#assert sns.__version__ == '0.12.2', sns.__version__
import matplotlib as mpl
COLOR = 'black'
mpl.rcParams['text.color'] = COLOR
mpl.rcParams['axes.labelcolor'] = COLOR
mpl.rcParams['xtick.color'] = COLOR
mpl.rcParams['ytick.color'] = COLOR


sns.set(rc={'axes.facecolor':'white', 'figure.facecolor':'white',
           'grid.color':'grey', 'grid.alpha':0.5})

tgtc = ['b','g','r','brown']


from glob import glob
from pathlib import Path
d = '/home/demitau/data_Quentin/pilot_studies/context_change_behav/'
fnbs = []
for fnf in glob(d + '2023-SE1-0*[0-9].log'):
    name = Path(fnf).name
    fnb = name.replace('.log','')
    fnbs += [fnb]
    #print( f'"{fnb}",' )
    
display(list(sorted(fnbs)))
    
#fnbs = ['2023-SE1-003_context_change_20230322_092232']

#fnbs = ['2023-SE1-012_context_change_20230417_130943']  # bad
fnbs = ['2023-SE1-011_context_change_20230417_130943']
print(fnbs)  

In [ ]:
dfs = []
for fnb in fnbs:
    fn = fnb + '.log'
    fnp = fnb + '.param'

    with open(pjoin(d,fn), 'r') as f:
         l = f.readline()
         truelen = len( l.split(',') )

    r = ('trial_index, current_phase_trigger, tgti_to_show,'
       ' vis_feedback_type, trial_type, special_block_type, block_ind, '
        ' feedbackX, feedbackY, unpert_feedbackX, unpert_feedbackY,'
         ' error_distance, target_coordX, target_coordY, '
         'feedbackX_when_crossing, feedbackY_when_crossing, '
         'jax1, jax2, reward, time')
    r = r.replace(' ','')
    colnames = r.split(',')
    print('len(colnames) = ',len(colnames),'truelen = ',truelen,colnames)

    nbad = 3  # recompense strip
    #colnames = ['trial_index', 'current_phase_']
    df = pd.read_csv(pjoin(d,fn), skipfooter = nbad, on_bad_lines='warn', header=0,
                    names=colnames, encoding='latin1')
    #encoding='latin-1'
    subj = fnb.split('_')[0]    
    df['subject'] = subj
    dfs += [df]

In [ ]:
df_unproc = pd.concat( dfs ).reset_index()
df_unproc = df_unproc.drop(labels=['index'],axis=1)
#df_unproc

from behav_proc import readParamFiles, addBasicInfo, getGeomInfo
fnp = fnb + '.param'
params, phase2trigger, trigger2phase, stage2pars  = readParamFiles(fnp, d)
home_position, target_coords = getGeomInfo(params)
hitr = float(params['radius_target']) + float(params['radius_cursor']) / 2

dfcs = []; dfccs = []; dfcpcs = []; dfctcs = [];    
for df in dfs:
    print('---------------  Starting processing new df')                    
    dfc, dfcc, dfcpc, dfctc = addBasicInfo(df, phase2trigger, params,
                                         home_position, target_coords)


    dfcc['nonhit'] = dfcc['error_distance'] > hitr
    dfcs += [dfc]; dfccs += [dfcc]; dfcpcs += [dfcpc]; dfctcs += [dfctc];   

dfcc_all  = dfcc
dfc_all   = dfc
dfcpc_all = dfcpc
dfctc_all = dfctc

subjects = dfcc_all['subject'].unique()
print(subjects)

subj = subjects[0]
print(subj)
dfccos  = dfcc_all.query('subject == @subj')
dfcos   = dfc_all.query('subject == @subj')
dfcpcos = dfcpc_all.query('subject == @subj')
#dfcc = dfccos
##########################
a = dfcc_all['error_area_signed'].abs()
thr = a.quantile(0.05); print(thr)
dfcc_all['error_area_signed_nonhit'] = a > thr

a = dfcc_all['error_area_signed_scaled_ed'].abs()
thr = a.quantile(0.05); print(thr)
dfcc_all['error_area_signed_scaled_ed_nonhit'] = a > thr

a = dfcc_all['error_lh_ang'].abs()
thr = a.quantile(0.05); print(thr)
dfcc_all['error_lh_ang_nonhit'] = a > thr

a = dfcc_all['error_area_signed_nn_scaled_ed'].abs()
thr = a.quantile(5e-2); print(thr)
dfcc_all['error_area_signed_nn_scaled_ed_nonhit'] = a > thr

a = dfcc_all['error_area2_signed_nn_scaled_ed'].abs()
thr = a.quantile(5e-2); print(thr)
dfcc_all['error_area2_signed_nn_scaled_ed_nonhit'] = a > thr
##############################
assert 'error' not in dfcc_all.columns
df_all = dfcc_all.drop(labels=['index', 'level_0'],axis=1)  #.copy()
#df_all['non_hit_not_adj'] = df_all['nonhit']
df_all['target_inds']     = df_all['tgti_to_show']
df_all['trials']          = df_all['trial_index']

# for err sens calc
#df_all['error']          = df_all['error_area_signed']
##############################
subjects_ = list( df_all['subject'].unique() )
print(subjects_)



In [ ]:
from behav_proc import computeErrSensVersions
#envs_cur = ['stable', 'random', 'all']
envs_cur = ['all']
#tgt_inds_cur = list( dfccos['tgti_to_show'].unique() )  + [None]
tgt_inds_cur =[None]
#tgt_inds_cur = [None]  # DEBUG

block_names_cur = ['all']
pertvals_cur = [None]
gseqcs_cur = [ None ]
dists_rad_from_prevtgt_cur = [None]
dists_trial_from_prevtgt_cur = [None]
error_type = 'MPE'
#time_lockeds=['target', 'feedback', 'trial_end']
time_lockeds=['target']

df_all['environment'] = 1111
#df_all['feedbackX'] = 1
#df_all['feedbackY'] = 1
#%debug
coln_pairs = [('error_area_signed_scaled_ed','error_area_ofb_signed_scaled_ed',
               'error_area_signed_scaled_ed_nonhit','signed_area_scaled_ed'),
           ('error_area_signed','error_area_ofb_signed','error_area_signed_nonhit','signed_area'),
              ('error_lh_ang','error_unpert_lh_ang','error_lh_ang_nonhit','lh_ang')]

coln_pairs = coln_pairs[0:1]

coln_pairs = [ ('error_area_signed_tln_scaled_ed','error_area_ofb_signed_tln_scaled_ed',
               'error_area_signed_scaled_ed_nonhit','signed_area_tln_scaled_ed') ]
##

coln_pairs = [ ('error_area_signed_nn_scaled_ed','error_area_ofb_signed_nn_scaled_ed',
               'error_area_signed_nn_scaled_ed_nonhit','signed_area_nn_scaled_ed') ]
coln_pairs += [('error_lh_ang','error_unpert_lh_ang','error_lh_ang_nonhit','lh_ang')]

##
coln_pairs = [ ('error_area2_signed_nn_scaled_ed',
                None,
               'error_area2_signed_nn_scaled_ed_nonhit',
                'signed_area2_nn_scaled_ed') ]

coln_pairs = [('error_lh_ang','error_unpert_lh_ang','error_lh_ang_nonhit','lh_ang')]

from base2 import areaBetweenTraj, calcNormCoefSectorArea
norm_coef = calcNormCoefSectorArea(params)
def trajPair2corr(dftraj1, dftraj2):
    return areaBetweenTraj(dftraj1, dftraj2, home_position,                                                                                                  
                target_coords, params ) * norm_coef

dfouts = []
import sys, traceback
try:
    for coln_error,coln_correction_calc,coln_nh, calc_name  in coln_pairs:
        for time_locked in time_lockeds:
            df_all_multi_tsz, ndf2vn = computeErrSensVersions(df_all, envs_cur,block_names_cur,
                pertvals_cur,gseqcs_cur,tgt_inds_cur,
                dists_rad_from_prevtgt_cur,dists_trial_from_prevtgt_cur,
                subj_list = subjects_, error_type=error_type,
                trial_shift_sizes = [1],
                addvars=[], target_info_type = 'locs',                                        
                computation_ver = 'computeErrSens3', 
                coln_nh = coln_nh,
                time_locked = time_locked,
                coln_nh_out = coln_nh + '_shifted',
                coln_error =coln_error,                                                                              
                coln_correction_calc = coln_correction_calc,                
                trajPair2corr = trajPair2corr,
                df_fulltraj = dfc,
                verbose=1,)
            df_all_multi_tsz['error_data_for_calc'] = calc_name
            dfouts += [df_all_multi_tsz]
    df_all_multi_tsz = pd.concat(dfouts).reset_index()
    
except Exception as e:
    print('EXC',e)
    exc_info = sys.exc_info()
    exc = traceback.TracebackException(*exc_info, capture_locals=True)
    ei = exc_info[2]

    #display(ei.tb_frame)
    psf = ei
    
    lfprev = None; lf = None
    lfs = []
    while psf.tb_frame is not None:
        stackframe = psf.tb_frame
        display(stackframe)
        lfprev = lf
        lf = stackframe.f_locals
        lfs += [lf]
        psf = psf.tb_next    
        if psf is None:
            break    

    print('EXC',e)

if coln_correction_calc is not None:
    df_all_multi_tsz['coln_corrc'] = df_all_multi_tsz[coln_correction_calc]
df_all_multi_tsz['coln_error'] = df_all_multi_tsz[coln_error]

bigEStis = df_all_multi_tsz.query('err_sens > 10')['trial_index'].to_numpy()
display(bigEStis)
bigEStis_neg = df_all_multi_tsz.query('err_sens < -10')['trial_index'].to_numpy()
#display(bigEStis_neg)

# plots

In [ ]:
tl = 'target'
edfc = calc_name
subj = subjects[0]

grp_perti = dfcos.groupby(['trial_index'])

pause_inds = dfcpc_all.query(f'subject == "{subj}"').\
    groupby('trial_index').max('time').index
#['trial_index']
pause_inds = np.array(pause_inds)

from base2 import areaBetweenTraj
def trajPair2corr_plot(dftraj1, dftraj2, ax):
    return areaBetweenTraj(dftraj1, dftraj2, home_position,                                                                                                  
                target_coords, params, ax=ax ) * norm_coef

print(tl, edfc)
dfesqs = (f'subject == "{subj}"'
          ' and error_data_for_calc == @edfc'
          ' and trial_shift_size == 1 and trial_group_col_calc == "trials"'
          ' and time_locked == @tl')    
display( df_all_multi_tsz.query(dfesqs) )

In [ ]:
colscols = [['trialwb','perturbation','trial_type'],
            ['prev_trial_type'],
            ['error_endpoint_ang', 'error_lh_ang'],            
            ['error_distance','time_mvt', 'traj_length'],
            ['error_area_signed_nn'],
            ['error_area_ofb_signed_nn'],           
            [coln_correction_calc], [coln_error],
            [ 'correction', 'prev_error'],
           ['err_sens','prev_err_sens', 'nonhit']]

st = 100
n = 5
trial_inds_glob1 = np.arange(st, st + n*n)
trial_inds_glob2 = trial_inds_glob1 + 1

rngs = f'{trial_inds_glob1[0]}-{trial_inds_glob1[-1]}'; print(rngs)

N = len(trial_inds_glob1); print(N)

In [ ]:
rngs

In [ ]:
from behav_proc import plotTraj, plotTraj2
from config2 import path_fig
#rng = range(600,603)
#rng = range(596,625)
#rng = range(8,10)
#rng = range(50,60)
#rng = range(55,57)

from behav_proc import row2multierr

show_dist_guides = False
ww = 7; hh =10
nc = int(np.ceil(np.sqrt(N) ) ); nr = int( np.ceil( N / nc ) )
#nr = N; nc = 2

fig,axs = plt.subplots(nr,nc, figsize=(nc*ww, nr * hh))
axsf = axs.flatten()

axi2d = {}
for axi,ti in enumerate(trial_inds_glob1):
    ax = axsf[axi]    
    if ti not in grp_perti.groups:
        if ti in pause_inds:            
            print(f'Skipping {ti} because pause')
            ax.set_title('PAUSE')
        else:
            print(f'Skipping {ti} because ?')
        continue
        
    df0 = dfcc.query(f'subject == "{subj}" and trial_index == {ti}')
    row = df0.to_dict('records')[0]
    
    ev = row['error_endpoint_ang']
    tiwb = row['trialwb']
    
    dfes = df_all_multi_tsz.query(dfesqs + f' and trial_index == {ti}')
    assert len(dfes) == 1
    row2 = dfes.to_dict('records')[0]
    row.update(row2)
    
    
    idx = grp_perti.groups[ti]
    dfcurtr = dfcos.loc[idx[1:]]  # zeroth is equal to zero
    import sys, traceback
    try:
#         plotTraj2(ax, dfcurtr, home_position, target_coords_homec, params,
#                 force_entire_traj=0, calc_area = 1, addinfo=row,
#                 titlecols = colscols, xlim=(-160,160))
        
        r = row2multierr(row, dfc, grp_perti, home_position,             
            target_coords, params, ax=ax,
            revert_pert = False, xlim=(-160,160),
            titlecols = colscols, addinfo=row)
        z = zip( ['err', 'error_lh_ang', 'error_unpert_lh_ang', 'start_dist',
                      'traja', 'trajoa', 'td',
                'length',  'enddist'], r)
        dd = dict(z)
    
    except Exception as e:
        print('EXC',e)
        exc_info = sys.exc_info()
        exc = traceback.TracebackException(*exc_info, capture_locals=True)

        psf = exc_info[2].tb_next
        stackframe = exc_info[2].tb_next.tb_frame

        display(psf.tb_frame)
        #display(psf.tb_next.tb_frame)

        local_vars_in_fun = stackframe.f_locals
        plt.close()
        break
    
    from matplotlib.ticker import (AutoMinorLocator, MultipleLocator)
    #ax.xaxis.set_major_locator(MultipleLocator(50))
    #ax.xaxis.set_major_locator(MultipleLocator(50))
    ax.set_yticks(np.arange(-50,400,50))
    #ax.xaxis.set_minor_locator(AutoMinorLocator(25))
    ax.grid(which='major', color='#CCCCCC', linestyle='--')
    #ax.grid(which='minor', color='#CCCCCC', linestyle=':')
    axi2d[axi] = {'df0':df0, 'row':row, 'dfcurtr':dfcurtr}
    
    print(f'error = {ev}, tiwb = {tiwb}')
    
plt.tight_layout()
plt.savefig(pjoin(path_fig, f'traj_2D_{row["subject"]}_{rngs}.pdf') )
plt.close()

In [ ]:
#ax = plt.gca()
correction = []
show_dist_guides = False
ww = 5; hh = 5

#L = min(nshow, len(trial_inds_glob1) -2 )
L = min(nshow, len(trial_inds_glob1)  )
nc = int(np.ceil(np.sqrt(L) ) ); nr = int( np.ceil( L / nc ) )
#nr = N; nc = 2

fig,axs = plt.subplots(nr,nc, figsize=(nc*ww, nr * hh))
axsf = axs.flatten()

axi = 0
#for trial_inds_glob1, valid_inds1, trial_inds_glob2
#for gti1,gti2 in list(zip(trial_inds_glob1, \
#                          trial_inds_glob2))[rng]:
for gti1,gti2 in list(zip(trial_inds_glob1, \
                          trial_inds_glob2))[:-1]:
    
    ax = axsf[axi]
    
    dftraj1 = dfc.query( 'trial_index == @gti1' )
    dftraj2 = dfc.query( 'trial_index == @gti2' )

    dftraj1 = dftraj1.loc[dftraj1.index[1: ] ]
    dftraj2 = dftraj2.loc[dftraj2.index[1: ] ]

    ab = trajPair2corr_plot(dftraj1, dftraj2, ax=ax  )

    correction_cur = ab
    correction += [correction_cur]
    #es = correction_cur/errors0[gti1]
    #s = ''
    #if es > 10:
    #    s = '*'
    #print(s, gti1,gti2,correction_cur,
    #      errors0[gti1], es )
    axi += 1

plt.tight_layout()
plt.savefig(pjoin(path_fig, f'doubletraj_2D_{subj}_{rngs}.pdf') )
plt.close()
# when first is on the right of second, then we have positive
# but to be similar to Romain's version we would like to have neg
# or not?

In [ ]:
dfccos['error_endpoint_ang']

In [ ]:
set(ynames) < set(df_.columns)

In [ ]:
'error_aug3' in df_.columns

In [ ]:
len(df_['error_aug3'].isna())

In [ ]:
numtrain = int(params['num_training'])

In [ ]:
os.pwd()

In [ ]:
pd.read_csv?

In [ ]:
df_ = dfcc.query('trial_index > 12 and trial_type == "perturbation"')
df_['time_lh'].describe()

In [ ]:
sns.lineplot(data = df_, y='time_lh', x='trialwb')

# Errors

In [ ]:
qs_notspec_not_afterpause = ('trial_index > @numtrain and trial_type != "error_clamp"'
       ' and prev_trial_type != "pause"')

qs_notspec_not_sandwich_not_afterpause = ('trial_index > @numtrain and trial_type != "error_clamp"'
        ' and special_block_type != "error_clamp_sandwich" and prev_trial_type != "pause"')

qs_notspec = ('trial_index > @numtrain and trial_type != "error_clamp"'
        ' and special_block_type != "error_clamp_sandwich"')

qs_notEC = ('trial_index > @numtrain and trial_type != "error_clamp"')

spectrials = ["error_clamp", "pause"]

qs_notspec_not_afterspec = ('trial_index > @numtrain and trial_type != "error_clamp"'
                   ' and prev_trial_type not in @spectrials')


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

grp = dfccos.query(qs_notEC).groupby('block_ind')
block_ends = grp['trial_index'].max()
# error -- angular signed
#'error_eucl',
#for yname in ['error_aug2', 'error_intdist2', 'error_intdist2nn','error_intdist','error_distance',              
#             'error_distance_lh']:
df_ = dfccos.query('trial_index > @numtrain').copy()
df_['traj_length_'] = df_['traj_length'] - df_['traj_length'].mean()

ynames = ['error_endpoint_ang', 'error_distance',
          'error_area2_signed_nn',
          coln_error,
          'time_lh', 'traj_length_']



#ynames = ['error','error_pert_adj']

ww = 20; hh=3
nr = len(ynames)
fig,axs =  plt.subplots(nr,1,figsize=(ww, nr*hh))
axs = axs.flatten()
for axi, yname in enumerate(ynames):    
#for yname in ['error_intdist2nn']:
    #yname = 'error_intdist2'
    #dfcc['tmp']
    #plt.figure(figsize=(20,3))
    ax = axs[axi]    
    print(yname)
    sns.lineplot(df_, x='trial_index', y =yname, ax=ax)
    #sns.lineplot(dfcc, x='trial_index', y ='perturbation', alpha=0.4, c='r')

    from config2 import path_fig
    
    mult = df_[yname].max() / 40
    #if yname == 'error_intdist2nn':
    #    mult = dfcc[yname].max()
    #else:
    #    mult = 1.            
        
    ax.plot(dfccos['trial_index'], dfccos['perturbation']._values * mult,
             alpha=0.4, c='r', label='pert')
    ax.vlines(pause_inds, -20 * mult, 20 * mult, color='green', ls = ':',
             label='pause')
    
    ax.vlines(block_ends, -20 * mult, 20 * mult, color='grey', ls = '--',
             label='block')

    ax.plot(dfccos['trial_index'], dfccos['tgti_to_show'] * 7 * mult, 
            color='cyan', alpha=0.5,
           label='target', ls=':')
    ax.grid(False)
    ax.axhline(y=0, ls=':', c='red')
    
    #plt.savefig( pjoin(path_fig, f'{subj}_{yname}.pdf' ) )
    #plt.close()
    if axi == nr - 1:
        ax.legend(loc='lower right')
plt.tight_layout()
plt.savefig( pjoin(path_fig, f'{subj}_{",".join(ynames)}.pdf' ) )
plt.close()

In [ ]:
dfext_ = dfccos.query(qs_notEC)
print(len(dfext_))
fg = sns.catplot(dfext_, kind='violin', x='prev_trial_type', 
                 y=coln_error, col = 'special_block_type')

In [ ]:

#!= "error_clamp" and prev_trial_type != "pause"
df_ = dfccos.query(qs_notspec)
print(len(df_))
fg = sns.relplot(df_, kind='line', x='trialwb', y=coln_error,
                 col='vis_feedback_type', 
            row='tgti_to_show', hue='nonhit')
for ax in fg.axes.flatten():
    ax.axhline(0,ls=':')

In [ ]:
df_ = dfccos.query(qs_notspec_not_afterspec)
grp = df_.groupby('block_ind')

df1 = grp['trial_index'].min().to_frame().rename(columns={'trial_index':'block_start_trial_index'})
df2 = grp['trial_index'].max().to_frame().rename(columns={'trial_index':'block_end_trial_index'})
block_bounds = pd.concat( [df1, df2], axis=1)

sts = block_bounds.query('block_ind >= 0')['block_start_trial_index']._values
df_block_start = dfccos.loc[dfccos['trial_index'].isin(sts)]

ends = block_bounds.query('block_ind >= 0')['block_end_trial_index']._values
df_block_end = dfccos.loc[dfccos['trial_index'].isin(ends)]

ynames = ['error_endpoint_ang', coln_error]
#'error', 


df_block_start = df_block_start[['block_ind', 'trial_type'] + ynames].set_index('block_ind')
#ynames_mod = [yn + '_start' for yn in ynames]
#rd = dict(zip(ynames,ynames_mod))
#df_block_start = df_block_start.rename(columns=rd)
df_block_start['type'] = 'start'

df_block_end =   df_block_end[['block_ind', 'trial_type'] + ynames].set_index('block_ind')
df_block_end['type'] = 'end'

df_startend = pd.concat([df_block_start, df_block_end])

for yname in ynames:
    plt.figure()
    ax = sns.violinplot(df_startend, y=yname, x='trial_type', hue='type', split=True)
    ax.axhline(0,ls=':', c='red')

In [ ]:
# define prev block ind

In [ ]:
df_ = dfccos.query(qs_notspec_not_afterpause)

for yname in ynames:
    plt.figure()
    ax = sns.violinplot(df_, y=yname, hue = 'trial_type', x='special_block_type',
                        split=True)
    ax.axhline(0,ls=':', c='red')

# ES

In [ ]:
from behav_proc import truncateDf
coln = 'err_sens'
dfc_multi_tsz = truncateDf(df_all_multi_tsz, coln, q=0.05, infnan_handling='discard',
                          cols_uniqify = ['trial_shift_size','trial_group_col_calc','error_data_for_calc'])
dfc_multi_tsz = dfc_multi_tsz.query('dist_rad_from_prevtgt2 != "nan"')

In [ ]:
calc_names = [calc_name]

In [ ]:
coln_pairs

In [ ]:
xlim = 0,450
dfr_all = []
#df_to_plot = df_all_multi_tsz
df_to_plot = dfc_multi_tsz

#' and special_block_type != "error_clamp_sandwich"'
qs0 = ('trial_shift_size == 1 and trial_group_col_calc == "trials"'
        ' and trial_type != "error_clamp"'                      
             f' and subject == "{subj}"')
#qs0 = qs_notEC + ' and trial_shift_size == 1 + and trial_group_col_calc == "trials"'
#dfc0_ = df_to_plot.query(qs0) 
r = []
for cni, cn in enumerate(calc_names):
    #df_all_multi_tsz
    #df_to_plot = dfc_multi_tsz            
    qs = qs0 + f' and error_data_for_calc == "{cn}" '
    dfc_ = df_to_plot.query(qs)

    print(len(dfc_))

    dfc_ = dfc_.set_index('trials')
    if cni != 0:
        dfc_ = dfc_[['err_sens','correction']]
    dfc_ = dfc_.rename(columns={'err_sens': f'err_sens:{cn}', 'correction': f'correction:{cn}'})
    r += [ dfc_]

#[dfc0_.drop(labels=['err_sens','correction'], axis=1)]
dfr0 = pd.concat( r, axis=1)
print(len(dfr0))

dfr0 = dfr0.reset_index()

print( 'num nans = ', dfr0['trial_index'].isna().sum() )
dfr = dfr0[~dfr0['trial_index'].isna() ] 

dfr_all += [dfr]
#dfr['block_ind']
#dfr.query('trial_type != "error_clamp"').groupby('block_ind')
#coln_pairs

ynames = []
for yn1, yn2, nh, cn in coln_pairs:
    ynames += [f'err_sens:{cn}', f'correction:{cn}']
    if yn1 is not None:
        ynames += [ yn1] ; 
    if yn2 is not None:
        ynames += [yn2]
ynames += ['error_endpoint_ang']
print(ynames)

#assert len(dfc_) == len(dfr), (len(dfc_), len(dfr))
#dfr[['trial_index','err_sens:signed_area','err_sens:lh_ang']]
dfcpcos_, dfcos_ = dfcpc_all.query('subject == @subj'), \
    dfc_all.query('subject == @subj')

assert 0 == sum( dfr['trial_index'].duplicated() )
# dfc_ = dfr.query('trial_shift_size == 1 and trial_group_col_calc == "trials" '
#         'and error_data_for_calc == @cn and trial_index < @xlim[1] and subject == @subj')
dfr_ = dfr.query('trial_index < @xlim[1]')

from behav_proc import multiplot
#['err_sens', 'correction']
multiplot(dfr_, dfcpcos_, dfcos_, ynames, start_ind=0, 
          xlim = xlim , ww=14, hh=1.5,
         show_titles = True)
plt.tight_layout()
#plt.savefig( pjoin(path_fig, f'{subj}_{",".join(ynames)}.pdf' ) )
plt.savefig( pjoin(path_fig, f'{subj}_nr={len(ynames)}_xlim={xlim}.pdf' ) )
plt.close()
    
dfr_all = pd.concat(dfr_all).reset_index()

In [ ]:
# allow only context shifts
dfes = dfc_multi_tsz.query(qs_notspec_not_sandwich_not_afterpause +\
            'and trial_shift_size == 1')# and err_sens >= -20 and err_sens <= 20')
print(len(dfes))
display(dfes['err_sens'].describe())
fg = sns.catplot(dfes, kind='violin',y='err_sens', x = 'error_data_for_calc')
for ax in fg.axes.flatten():
    ax.set_ylim(-10,10)

In [ ]:
fg = sns.relplot(dfes, kind='line', x='trialwb', y='err_sens', 
                 col='vis_feedback_type', 
            row='tgti_to_show')
for ax in fg.axes.flatten():
    ax.axhline(0,ls=':',color='red')

In [ ]:
dfes = dfes.copy()
dfes['abs_err_sens']  = np.abs(dfes['err_sens'])

In [ ]:
fg = sns.relplot(dfes, kind='line', x='trialwb', y='abs_err_sens', 
                 col='vis_feedback_type', 
            row='tgti_to_show')
for ax in fg.axes.flatten():
    ax.axhline(0,ls=':',color='red')

In [ ]:
fg = sns.relplot(dfes, kind='line', x='trialwb', y='err_sens', hue='nonhit')
for ax in fg.axes.flatten():
    ax.axhline(0,ls=':',color='red')
    ax.set_title(edfc)

In [ ]:
df_block_start

In [ ]:
df_ = dfes
grp = df_.groupby('block_ind')

df1 = grp['trial_index'].min().to_frame().rename(columns={'trial_index':'block_start_trial_index'})
df2 = grp['trial_index'].max().to_frame().rename(columns={'trial_index':'block_end_trial_index'})
block_bounds = pd.concat( [df1, df2], axis=1)

sts = block_bounds.query('block_ind >= 0')['block_start_trial_index']._values
df_block_start = dfes.loc[dfes['trial_index'].isin(sts)]

ends = block_bounds.query('block_ind >= 0')['block_end_trial_index']._values
df_block_end = dfes.loc[dfes['trial_index'].isin(ends)]

ynames = ['err_sens']
#'error', 


df_block_start = df_block_start[['block_ind', 'trial_type'] + ynames].set_index('block_ind')
#ynames_mod = [yn + '_start' for yn in ynames]
#rd = dict(zip(ynames,ynames_mod))
#df_block_start = df_block_start.rename(columns=rd)
df_block_start['type'] = 'start'

df_block_end =   df_block_end[['block_ind', 'trial_type'] + ynames].set_index('block_ind')
df_block_end['type'] = 'end'

df_startend = pd.concat([df_block_start, df_block_end])

for yname in ynames:
    plt.figure()
    ax = sns.violinplot(df_startend, y=yname, x='trial_type', hue='type', split=True)
    ax.axhline(0,ls=':', c='red')
    ax.set_ylim(-10,10)

In [ ]:
display( df_startend.groupby('type').mean(numeric_only=1) )
display( df_startend.groupby('type').std(numeric_only=1) )

In [ ]:
from config2 import path_code

In [ ]:
filename = pjoin(path_code,'exper_protocol',
                 'dataMEG/Dmitrii_test_context_change_20230425_174656')

In [ ]:
trial_dur_expected = 4
nread = int( params['FPS'] ) * trial_dur_expected * 5

In [ ]:
from exper_protocol.utils import getLastInfo
getLastInfo(filename, nread)

In [ ]:
grp_pertim.columns

In [ ]:
last_trial_ind-1, last_trial_block_ind

In [ ]:
rwd_until = df

In [ ]:
last_trial_ind, last_trial_ind, last_trial_block_ind